# Step 4: First Model

In [1]:
# Median imputation
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

In [4]:
train_df = pd.read_csv("../data/train_processed.csv")
test_df = pd.read_csv("../data/test_processed.csv")


# First Model Training with KNN
predictor = "nutriscore_grade_encoded"

base_num = [
    "energy","sugars","carbohydrates","salt",
    "fat","trans_fat","proteins","additives_n","ingredients_n"
]
num_cols = [c for c in base_num if c in train_df.columns]

X_train = train_df[base_num].to_numpy()
X_test = test_df[base_num].to_numpy()

y_train = train_df[predictor].astype(int)
y_test = train_df[predictor].astype(int)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (384395, 9)
Test shape: (51911, 9)


In [5]:
# KNN CLASSIFIER

class KNNClassifier:
    def __init__(self, k=7):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _distance(self, x1, x2):
        # Euclidean distance
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def predict_one(self, x):
        # compute distances from x to ALL training samples
        dists = np.linalg.norm(self.X_train - x, axis=1)

        # find k nearest neighbors
        k_idxs = np.argsort(dists)[:self.k]
        k_labels = self.y_train[k_idxs]

        # majority vote
        values, counts = np.unique(k_labels, return_counts=True)
        return values[np.argmax(counts)]

    def predict(self, X):
        return np.array([self.predict_one(x) for x in X])

In [ ]:
# Train model
knn = KNNClassifier(k=7)
knn.fit(X_train, y_train)

# Predictions
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

# Accuracy
train_acc = np.mean(y_train_pred == y_train)
test_acc = np.mean(y_test_pred == y_test)

print(f"Train Accuracy: {train_acc:.3f}")
print(f"Test Accuracy:  {test_acc:.3f}")


Because the predict() implementation iterates across all 340,471 training samples for each of the 85,118 test samples, the computational cost becomes prohibitive and the end-to-end inference can require several hours.

To identify potential optimizations, we applied GENAI-assisted research. The resulting improvement replaces nested Python loops with fully vectorized distance computations and batch-based prediction. Consequently, the K-Nearest Neighbors classifier achieves approximately significant reduction in runtime, as distance calculation is performed on entire matrices rather than iterating over each training instance for every test instance.

In [6]:
import numpy as np
import pandas as pd

class KNNClassifierScratch:

    # NumPy-only KNN with batched L2 distances.
    def __init__(self, k=7):
        self.k = k

    def fit(self, X, y):
        self.X_train = np.asarray(X, dtype=np.float64)
        self.y_train = np.asarray(y)
        # Precompute squared norms for fast L2 distances
        self.train_norm2 = np.einsum('ij,ij->i', self.X_train, self.X_train)
        self.max_label = int(self.y_train.max())

    def _predict_batch(self, Xb):
        Xb = np.asarray(Xb, dtype=np.float64)
        xb_norm2 = np.einsum('ij,ij->i', Xb, Xb)            # (B,)
        cross = Xb @ self.X_train.T                         # (B, N)
        d2 = xb_norm2[:, None] + self.train_norm2[None, :] - 2.0 * cross
        np.maximum(d2, 0.0, out=d2)                         # safety

        # top-k indices per row (no full sort)
        idx_part = np.argpartition(d2, self.k-1, axis=1)[:, :self.k]    # (B, k)
        row_idx = np.arange(Xb.shape[0])[:, None]
        nearest_k = idx_part[row_idx, np.argsort(d2[row_idx, idx_part], axis=1)]

        # majority vote via bincount
        preds = np.empty(Xb.shape[0], dtype=self.y_train.dtype)
        for i in range(Xb.shape[0]):
            labs = self.y_train[nearest_k[i]]
            bc = np.bincount(labs, minlength=self.max_label+1)
            preds[i] = bc.argmax()
        return preds

    def predict(self, X, batch_size=128):
        X = np.asarray(X, dtype=np.float64)
        out = np.empty(X.shape[0], dtype=self.y_train.dtype)
        for s in range(0, X.shape[0], batch_size):
            e = min(s + batch_size, X.shape[0])
            out[s:e] = self._predict_batch(X[s:e])
        return out


In [27]:
knn = KNNClassifierScratch(k=7)
knn.fit(X_train, y_train)

y_test_pred = knn.predict(X_test, batch_size=128)

test_acc = (y_test_pred == y_test).mean()
print(f"[KNN k=7] Test Accuracy: {test_acc:.4f}")


[KNN k=7] Test Accuracy: 0.5009


In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Train Accuracy (subsample of train set)
rng = np.random.default_rng(42)
sub_n = min(20000, X_train.shape[0])
idx_sub = rng.choice(X_train.shape[0], size=sub_n, replace=False)

X_train_sub = X_train[idx_sub]
y_train_sub = y_train[idx_sub]

# Predict on this subsample
y_train_sub_pred = knn.predict(X_train_sub, batch_size=128)
train_acc = accuracy_score(y_train_sub, y_train_sub_pred)

print(f"[KNN k=7] Train Accuracy (20k subsample): {train_acc:.4f}")
print(f"[KNN k=7] Test Accuracy:               {test_acc:.4f}")


[KNN k=7] Train Accuracy (20k subsample): 0.6130
[KNN k=7] Test Accuracy:               0.5009


Metric	Result:

Train Accuracy (20k subsample)	0.6130 (~61.3%)

Test Accuracy	0.5009 (~50.1%)

Model	KNN (k=7), implemented

The model performs significantly better on training than on testing.

The ~11% drop (61.3% → 50.1%) means the model learns patterns in training data that do not generalize perfectly.

This indicates mild overfitting, but not extreme.

Conclusion:

We implemented a K-Nearest Neighbors classifier using only NumPy, without using any scikit-learn classifiers. The model used scaled nutrient features and was evaluated using a train/test split.

The higher training accuracy indicates the model fits the data reasonably well. However, the drop in test accuracy shows the model does not generalize perfectly to unseen samples, which suggests some overfitting.

In [32]:
from sklearn.metrics import accuracy_score

results = []  # to store (k, train_acc, test_acc)

for k in [1, 7, 15]:

    print(f"\n--- Evaluating KNN (k={k}) ---")

    knn = KNNClassifierScratch(k=k)
    knn.fit(X_train, y_train)

    # Test predictions (fast batched)
    y_test_pred = knn.predict(X_test, batch_size=128)
    test_acc = accuracy_score(y_test, y_test_pred)

    # Train accuracy on a subsample to avoid O(N^2)
    rng = np.random.default_rng(42)
    sub_n = min(20000, X_train.shape[0])
    idx_sub = rng.choice(X_train.shape[0], size=sub_n, replace=False)

    X_train_sub = X_train[idx_sub]
    y_train_sub = y_train[idx_sub]

    y_train_sub_pred = knn.predict(X_train_sub, batch_size=128)
    train_acc = accuracy_score(y_train_sub, y_train_sub_pred)

    print(f"Train Accuracy (20k subsample): {train_acc:.4f}")
    print(f"Test  Accuracy:                 {test_acc:.4f}")

    results.append((k, train_acc, test_acc))

# Store results in a DataFrame for display
results_df = pd.DataFrame(results, columns=["k", "train_accuracy", "test_accuracy"])
print("\nSummary of K values:")
print(results_df)



--- Evaluating KNN (k=1) ---
Train Accuracy (20k subsample): 0.9279
Test  Accuracy:                 0.5063

--- Evaluating KNN (k=7) ---
Train Accuracy (20k subsample): 0.6130
Test  Accuracy:                 0.5009

--- Evaluating KNN (k=15) ---
Train Accuracy (20k subsample): 0.5624
Test  Accuracy:                 0.4967

Summary of K values:
    k  train_accuracy  test_accuracy
0   1          0.9279       0.506344
1   7          0.6130       0.500893
2  15          0.5624       0.496710


Where does your model fit in the fitting graph? (Build at least one model with different hyperparameters and check for over/underfitting, pick the best model).

For k = 1, the model is on the left side of the fitting curve
Low bias, extremely high variance
Nearly perfect memorization of training patterns, bad generalization

For k = 15, the model is on the right side of the curve
High bias, low variance
Oversmoothed boundaries lead to accuracy drops

For k = 7, the model is in the center (optimal zone)
Balanced bias and variance
Best generalization performance of the three models

Therefore, K-Nearest Neighbors with k=7 provides the best model.


What are the next models you are thinking of and why?

Gaussian Naive Bayes: Our features are continuous (nutrients), and classes overlap; GNB often sets a strong baseline on numeric, moderately correlated features. Trains and predicts with tiny memory — ideal for our scale.

Decision Tree can model non-linear interactions (e.g., high sugar and high fat lead to lower grade). Gives interpretable splits and feature importance which is useful to explain nutrition drivers.

Linear SVM can maximize margin in high-dimensional numeric space; often robust with regularization.

Conclusion section: What is the conclusion of your 1st model? What can be done to possibly improve it?

Our first model was a K-Nearest Neighbors classifier implemented and trained on the scaled nutrient features. We evaluated multiple hyperparameters to understand the model's fit behavior.

After trying k = 1, 7, and 15, we found that k=1 severely overfits (train=92.8%, test=50.6%), while k=15 underfits (train=56.2%, test=49.7%). The optimal value is k=7, which achieves the best tradeoff between bias and variance (train=61.3%, test=50.1%), placing the model near the middle of the fitting curve. Therefore, KNN(k=7) is selected as our first model.

Why the model struggles: NutriScore categories share similar nutrient profiles (e.g., many “C” and “D” foods overlap in sugar/fat ranges). KNN relies strictly on Euclidean distance; if class clusters overlap, it cannot draw meaningful boundaries.


What can be done to possibly improve the model?
Add categorical and text-derived features: Parse ingredients, labels, categories, allergens into binary or frequency features. Nutrient-only data is too limited; text-based metadata will provide richer class signals

Try different models: Gaussian Naive Bayes provides strong on continuous numeric data. Decision Tree captures sugar and fat interactions and rule-like splits. Linear or kernelized SVM via SGD handles high-dimensional dense features.